GWAS解析　体験手引書
参考図書　がんゲノムデータ解析　著者：清水厚志、坊農秀雅　メディカル・サイエンス・インターナショナル

手順1
Gドライブの直下にprogramsフォルダを作成
programsフォルダの直下にWDフォルダを作成
WDフォルダの直下に、BIN、QC、GWAS、DATASETフォルダを作成
WD - BIN
   ├ DATASET
   ├ GWAS
   └ QC

手順2
この実行ファイルはWDフォルダの直下に保存する
WD - BIN
   ├ DATASET
   ├ GWAS
   ├ QC
   └ KC_Learning.ipynb

手順3
plinkのHPに移動して、PLINK1.9の最新バージョン(windows 64bit)及び plink2.0の最新バージョンをダウンロードすr
plinkのHPからダウンロードすると、「plink.exe」 「plink2.exe」があるので、このexeファイルをWDフォルダ配下の「WD」フォルダへ移動する。
WD - BIN
   │ ├ plink.exe
   │ └ plink2.exe
   ├ DATASET
   ├ GWAS
   ├ QC
   └ KC_Learning.ipynb

##############################
6.4章　データの入手
##############################

GWASはジェノタイプデータと対象となる形質との間で関連解析を行う。
そのため、ジェノタイプデータと形質データをそれぞれ準備する必要がある。
形質データには、GWASの対象となる形質（目的変数）の他に、その目的変数に影響を与える遺伝形質以外の要因（共変数、または調整変数とも呼ぶ）のデータを含んでいる必要がある。

本稿ではジェノタイプデータと形質データを解析に使用できるように整形していく。
ここで利用するジェノタイプデータはDNAマイクロアレイで一部のバリアントのジェノタイプを決定したあと、インピュテーションという方法で、
その他の箇所のジェノタイプを統計的に推定データを模している。

インピュテーションにはスパコンレベルの計算力が必要となる。そのため、各研究機関が独自で行うのは現時点では効率的ではない。
企業や他機関がインピュテーション済みデータとして提供しているものを利用するケースが多くなると想定される。

インピュテーション
GWASで使われるジェノタイプデータは、DNAマイクロアレイを使って決定される。
DNAマイクロアレイでは数千万箇所に呼ぶSNPのうち、一般的にみられる100万箇所程度を対象に実験的にジェノタイプを決定する。
残りのSNPは数千人規模の全ゲノムシーケンスで得られた、ヒトのSNPの大半が網羅されたデータを「参照パネル」として使用し、
DNAマイクロアレイ由来のデータに含まれていない箇所のSNPのジェノタイプを推定する。
この推定は、「同じ染色体内のSNPは互いに連鎖しており、一般的に近傍であるほど組替えが起こりにくい」という法則を利用する。
ただし、実際に観察されるSNP間の組替えの頻度は単純に距離だけでは決まっておらず、組換えが比較的発生しづらい領域が存在する（連鎖不均衡）。
インピュテーションはこうした実際に観察される組替えの頻度情報などを加味してジェノタイプの補完を進める。

インピュテーションのツール
・SHAPEIT
・INPUTE2

In [ ]:
# 以下のリンクから、GWAS解析に使うデータをダウンロードする。
# http://www.medicalgenome.info/data/CancDAT/chapter6_GWAS/
# Linuxならコマンドラインでwgetを使う場面だが、KCの環境ではwindowsかつコマンドラインが使えないので、Python上で一括ダウンロードできるようにする
# 今回利用するジェノタイプデータセットは国際1000人ゲノム計画により得られた2,504人のジェノタイプデータのうち、日本人104人のデータに基づき、
# シミュレーションによって作成されたデータセットとなっている。つまり、個人情報ではない情報になっている。
# サンプルサイズはケースとコントロールでそれぞれ1000人、計2000人となっている。

# 最低限知っておくべきLinuxコマンド
# ../   一つ上のディレクトリ階層を示す
# ./    現在のディレクトリを示す
# cd    ディレクトリを変わる指示　　cd ../       とか   cd ./QC     みたいな使い方をする
# pwd   現在のディレクトリがわからなくなったら、これ。
# ls    現在のディレクトリの中身を表示させる

# 結構時間がかかります。

%cd ~/DATASET
! wget -N -r --no-parent -nd http://www.medicalgenome.info/data/CancDAT/chapter6_GWAS/

In [ ]:
# ディレクトリを変更する
%cd ../

# 現在地を示す。WDが望ましい
%pwd

In [ ]:
# ダウンロードしたファイルの中で、gzファイルがある。これがVCF。
# VCFファイルの中身を確認する。VCF構造の説明は「vcf構造.png」を参照のこと。
# M2 Mac Book Airで 17sec

import gzip
with gzip.open('./DATASET/1000GP_Phase3_chr1.gen.gz', 'rt', 'utf-8') as inst:
    for line in inst:
        print(line, end='')

In [ ]:
# データセットの確認
# gen.gzファイルにはジェノタイプ情報のみが保存されているが、sampleファイルにはgen.gzファイルに対応するサンプルのID名などの情報が格納されている。
# ヘッダは2行。1行目は列名、2行目は各列のデータの種類（型）を示している。（0, D,B, P or C）
# 0はID、Dは性別など２値で表される共変数、Bは病気の有など２値で表される目的変数、PまたはCはBMIなどの連続値で表される目的変数を示す。
# 一般的に個人IDはIDのペアで表す。実際には同じIDが入っていることが多い。本体は片方に家系IDを入れる。

! head ./DATASET/1000GP_Phase3_chr1.sample

In [ ]:
# データセットの確認
# 少し前に確認したものと同じ。これだと途中で切れてしまってわかりにくいかも。
# 最後の「1-20」は列数を示している。
# headは行数のコントロールで10行まで示すコマンド。
# 「|」はパイプと呼ばれるLinuxコマンドの特徴。複数のコマンドを組み合わせることができ、パイプを使った処理をパイプラインと呼ぶ。
# cutコマンドには、半角スペースでデータが区切られていることをダブルクオーテーションで区切った部分で教えている。

! gunzip -c ./DATASET/1000GP_Phase3_chr1.gen.gz | head | cut -d " " -f 1-20

In [ ]:
# ファイル形式の変換
# 実行時間　M2 Macbook Air 16m53s
# gen.gzファイルを、PLINKのバイナリ形式に変換する。このとき、.bed, .bim, .fam という３つのファイルが出来上がる。
# ジェノタイプデータは .bed
# バリアントの名前、位置、内容の情報は .bim
# サンプルのIDや性別、形質の情報は .fam

# plinkコマンドの意味
# --gen;                .gen.gz ファイルを指定する
# --sample;             .gen.gz に対応する sample ファイルを指定する
# --oxford-single-chr;  このファイルの染色体番号を指定する
# --make-bed;           バイナリplink形式で出力させる
# --keep-allele-order;  データ中のアレルの順番が入れ替えられることを防ぐ
# --out;                出力ファイル名を指定する

!./BIN/plink --gen ./DATASET/1000GP_Phase3_chr1.gen.gz --sample ./DATASET/1000GP_Phase3_chr1.sample --oxford-single-chr 1 --make-bed --keep-allele-order --out ./QC/1000GP_Phase3_chr1


In [ ]:
# .fam .bimのファイル構造を確認する

# サンプルのIDや性別、形質の情報は .fam
! head ./QC/1000GP_Phase3_chr1.fam 

In [ ]:
# バリアントの名前、位置、内容の情報は .bim
! head ./QC/1000GP_Phase3_chr1.bim